In [18]:
import os
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from pinecone import Pinecone

In [20]:
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_API_ENV = os.getenv('PINECONE_API_ENV')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
        )
    
    documents = loader.load()

    return documents

In [4]:
extracted_data = load_pdf("data/")

In [5]:
#Create text chunks
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap = 20
        )
def text_split(extracted_data):
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [6]:
text_chunks = text_split(extracted_data)

In [7]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [8]:
#download embedding model
embeddings = download_hugging_face_embeddings()

c:\Users\user\Desktop\BLOG-APP-1\travelbot\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [5]:
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

pinecone_instance = Pinecone(api_key=PINECONE_API_KEY, environment='gcp-starter') #I dont think this line does anything but sijaitoa just incase

index_name = "blogbot"

# Create a PineconeVectorStore instance with the Pinecone instance and index
""" from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name) """

' from langchain_pinecone import PineconeVectorStore\n\ndocsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name) '

In [9]:
def load_pinecone_existing_index():
    docsearch = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embeddings)
    return docsearch
docsearch=load_pinecone_existing_index()

query = "What is blogging"
docs=docsearch.similarity_search(query, k=3)
print("Result", docs[0].page_content)

Result of web pages, and a blog is a websit e wit h web pages present ed in reverse
chronological order. 
It's what  you  do wit h a blog t hat makes it  what ever you  wish it  to be. 
Blogging:  Blogging is t he modern t erm f or web pu blishing. If  you  publish
anyt hing on t he web, it  is called a post  and t he act ion is called blogging .
You cou ld be pu blishing on a sit e, blog, Facebook, St umbleUpon, Twit ter,
SnapChat , Inst agram, LinkedIn, it  doesn' t mat ter. Y ou are not


In [10]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just vaguely craft something, but dont use much facts.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [11]:

PROMPT=PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [22]:
llm=ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-3.5-turbo", 
    temperature=0.5
    )

In [23]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [24]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print(result['result'])

Blogging is the modern term for web publishing where you publish content on the web, referred to as a post.
